# Calculating the distance between the Customer's city and the Seller's city

In [9]:
from pyspark.sql import SparkSession, functions as F
spark = SparkSession.builder.getOrCreate()

orders_items_df = spark.read \
                 .option('escape', '\"') \
                 .option('quote', '\"') \
                 .csv('./dataset/olist_order_items_dataset.csv', header=True, multiLine=True, inferSchema=True)

orders_df = spark.read \
                 .option('escape', '\"') \
                 .option('quote', '\"') \
                 .csv('./dataset/olist_orders_dataset.csv', header=True, multiLine=True, inferSchema=True)

customers_df = spark.read \
                  .option('escape', '\"') \
                  .option('quote', '\"') \
                  .csv('./dataset/olist_customers_dataset.csv', header=True, multiLine=True, inferSchema=True)

sellers_df = spark.read \
                  .option('escape', '\"') \
                  .option('quote', '\"') \
                  .csv('./dataset/olist_sellers_dataset.csv', header=True, multiLine=True, inferSchema=True)

geo_df = spark.read \
                  .option('escape', '\"') \
                  .option('quote', '\"') \
                  .csv('./dataset/olist_geolocation_dataset.csv', header=True, multiLine=True, inferSchema=True)

data_df = orders_df.filter(F.col('order_status') == 'delivered').join(customers_df, 'customer_id')

data_df = orders_items_df.join(data_df, 'order_id') \
                         .join(sellers_df, 'seller_id') \
                         .select('customer_state', 'customer_city', 'customer_zip_code_prefix', 'seller_state', 'seller_city', 'seller_zip_code_prefix', 'freight_value')

data_df = data_df.join(geo_df, data_df.customer_zip_code_prefix == geo_df.geolocation_zip_code_prefix)
#                  .join(geo_df, data_df.seller_zip_code_prefix == geo_df.geolocation_zip_code_prefix)


# data_df = data_df.withColumn("a", ( \
#         F.pow(F.sin(F.radians(F.col("destination_latitude") - F.col("origin_latitude")) / 2), 2) + \
#         F.cos(F.radians(F.col("origin_latitude"))) * F.cos(F.radians(F.col("destination_latitude"))) * \
#         F.pow(F.sin(F.radians(F.col("destination_longitude") - F.col("origin_longitude")) / 2), 2))) \
#         .withColumn("distance", F.atan2(F.sqrt(F.col("a")), F.sqrt(-F.col("a") + 1)) * 12742000)

data_df.limit(5).toPandas()

,customer_state,customer_city,customer_zip_code_prefix,seller_state,seller_city,seller_zip_code_prefix,freight_value,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,SP,sao paulo,1238,SP,sao paulo,3225,7.47,1238,-23.541953,-46.659318,sao paulo,SP
1,SP,sao paulo,1238,SP,sao paulo,3225,7.47,1238,-23.544438,-46.651813,sao paulo,SP
2,SP,sao paulo,1238,SP,sao paulo,3225,7.47,1238,-23.542944,-46.657937,sao paulo,SP
3,SP,sao paulo,1238,SP,sao paulo,3225,7.47,1238,-23.543399,-46.651518,sao paulo,SP
4,SP,sao paulo,1238,SP,sao paulo,3225,7.47,1238,-23.543239,-46.657316,sao paulo,SP
